In [1]:
from pathlib import Path
from tqdm.notebook import tqdm

import numpy as np
import pandas as pd
import torch

import diffhydro as dh
import xtensor as xt

from diffhydro.io import read_rapid_graph, read_multiple_rapid_graphs

### Experiment parameters

In [2]:
root = Path("./data") # Set your data root path

In [3]:
discharge_path = root / "geoglows" / "retro_feather"
vpu_config_path = root / "geoglows" / 'configs'
runoff_path = root / "geoglows" / "daily_sparse_runoff.feather"
interp_weight_path = root / "geoglows" / "interp_weight.feather"

In [4]:
# graph partitioning parameters
plength_thr = 10**4
node_thr = 10**4
# Routing model parameterss
max_delay = 32
dt = 1/24
# Experiment pathes and variables
device = "cuda:7"

### Download data if necessary

If the dataset is not already downloaded, please make sure your root path points to a folder with sufficient disk space.

If you are only interesting in running the simulation with diffroute, then please leave the flage download_gt_as_well = False.
This will download everything needed to run the model (\~20 GB ~ 1 min.), and will not download the original simulation results (\~700GB).

Only if you are interested in validatig the results against the original data set download_gt_as_well = True.
This will additionally download the output river discharge, but make sure your root path points to a location with sufficient storage capacity (>700GB)

In [5]:
#import sys; sys.path.insert(0, "..") # Needed to access the download utilities
#from utils.download import download_full_geoglows_data
#download_full_geoglows_data(root, exclude_discharge=True)

In [6]:
pixel_runoff = xt.read_feather(runoff_path, dims=["time", "spatial"]).to(device=device) #, dtype=torch.half
pixel_runoff = pixel_runoff.sel(time=slice(None, "2019")).expand_dims("batch").transpose("batch", "spatial", "time") / (3600. * 24)
cat_interp_df = pd.read_feather(interp_weight_path).set_index("river_id")

In [7]:
#pixel_runoff.values = pixel_runoff.values.half()
#pixel_runoff["time"].dtype

### Run the full simulation and compute max Q stat

In [8]:
#vpus = [x for x in vpu_config_path.glob("*") if not x.name.startswith(".")]
#g = read_multiple_rapid_graphs(vpus, plength_thr=plength_thr, node_thr=node_thr).to(device)
%time g = pd.read_pickle("./gg.pkl").to(device)

CPU times: user 14 s, sys: 1.8 s, total: 15.8 s
Wall time: 15.7 s


In [9]:
cat = dh.StagedCatchmentInterpolator(g, pixel_runoff, cat_interp_df).to(device)#.half()
model = dh.LTIStagedRouter(max_delay=max_delay, dt=dt).to(device)

  0%|          | 0/729 [00:00<?, ?it/s]

In [10]:
cat_runoffs = cat.yield_all_runoffs(pixel_runoff.to(device))
cat_discharge = model.route_clusters_sequence(cat_runoffs, g)

for q in tqdm(cat_discharge):
    pass

0it [00:00, ?it/s]

### Validate against the original simulation

In [10]:
model = dh.LTIRouter(
              max_delay=max_delay,
              dt=dt,
          ).to(device)

nses = []

for i,vpu in enumerate(tqdm(list(discharge_path.glob("*.feather")))):
    print(f"Loading vpu {vpu.stem} data ...")
    q = xt.read_feather(vpu, dims=["time", "spatial"])\
        .transpose("spatial", "time")\
        .expand_dims("batch")
    
    g = read_rapid_graph(vpu_config_path / vpu.stem, 
                         plength_thr=plength_thr, 
                         node_thr=node_thr).to(device)
    
    interp_df = cat_interp_df.loc[g.nodes]
    pix_idxs = interp_df["pixel_idx"].unique()
    runoff = pixel_runoff.sel(spatial=pix_idxs)

    print(f"Routing vpu {vpu.stem} ...")
    
    cat = dh.StagedCatchmentInterpolator(g, runoff, interp_df).to(device)
    
    cat_runoffs = cat.yield_all_runoffs(runoff)
    cat_discharges =  model.route_clusters_sequence(cat_runoffs, g)

    vpu_nses = []
    for output in tqdm(cat_discharges):
        y = q.sel(spatial=output["spatial"]).to(device)
        nse = dh.nse_fn(output.values, y.values)
        vpu_nses.append(pd.Series(nse.squeeze().cpu().numpy(), 
                              index=output["spatial"].values.cpu().numpy()))
    nses.append(pd.concat(vpu_nses))

  0%|          | 0/125 [00:00<?, ?it/s]

Loading vpu 202 data ...
#### Upstream stats computations ... ####


Computing breakpoints:   0%|          | 0/34443 [00:00<?, ?it/s]

#### Segmentation into subgraphs ... ####
Removing edges...


  0%|          | 0/34443 [00:00<?, ?it/s]

...

Match breakpoint nodes across clusters...


  0%|          | 0/94 [00:00<?, ?it/s]

Routing vpu 408 ...


  0%|          | 0/94 [00:00<?, ?it/s]

0it [00:00, ?it/s]

In [15]:
pd.concat(nses).median()

np.float32(0.999668)